In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from scipy.sparse import lil_matrix, csr_matrix

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
EFFECT_SIZE_PATH = DATA_PATH.joinpath('capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023_gene_symbol_harmonized.pkl')
PROTEIN_SEQ_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')

In [2]:
df = pd.read_pickle(EFFECT_SIZE_PATH)

df = df.groupby(['gene_symbol_harmonized'], group_keys=False).apply(lambda x: x.loc[x['est_m_ea'] == x['est_m_ea'].max()])

assert not df['gene_symbol_harmonized'].duplicated().any()

df_effect_size = df.copy()

In [11]:
df = pd.read_pickle(PROTEIN_SEQ_PATH)

df = df.groupby(['gene_symbol_harmonized']).filter(lambda x: x['motifs'].notna().any())

df = df[['gene_symbol_harmonized', 'seq']]

df = df.drop_duplicates()

df = df_effect_size.merge(df, how='inner', on='gene_symbol_harmonized')

df = df[['gene_symbol_harmonized', 'est_m_ea', 'seq']]

print(df.shape)

assert not df[['gene_symbol_harmonized', 'seq']].duplicated().any()

# df.to_pickle(DATA_PATH.joinpath('protein_effect_size_analysis.pkl'))

df['class'] = df['est_m_ea'].apply(lambda x: 0 if x <= 0 else 1)

# df = df.groupby(['gene_symbol_harmonized']).sample(n=1)

hold_out_genes = df['gene_symbol_harmonized'].drop_duplicates().sample(frac=.5)

_df_validation = df.loc[~df['gene_symbol_harmonized'].isin(hold_out_genes)]

_df_holdout = df.loc[df['gene_symbol_harmonized'].isin(hold_out_genes)]

(2929, 3)


In [23]:
model = None
scores = []
for i in tqdm(range(0,1)):

    df = _df_validation.copy()

    train_genes = df['gene_symbol_harmonized'].drop_duplicates().sample(frac=.8)

    df = df.loc[df['gene_symbol_harmonized'].isin(train_genes)]

    min_count = df['class'].value_counts().min()

    df = df.groupby(['class']).sample(n=min_count)

    df_train = df.copy()

    df = _df_validation.copy()

    df = df.loc[~df['gene_symbol_harmonized'].isin(train_genes)]

    # min_count = df['class'].value_counts().min()

    # df = df.groupby(['class']).sample(n=min_count)

    df_valid = df.copy()

    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(42,42))
    vectorizer = vectorizer.fit(df_train['seq'].tolist())

    X_train = vectorizer.transform(df_train['seq'].tolist())
    y_train = df_train['class'].tolist()

    X_valid = vectorizer.transform(df_valid['seq'].tolist())
    y_valid = df_valid['class'].tolist()

    clf = KNeighborsClassifier(n_neighbors=3)

    clf = clf.fit(X_train, y_train)

    score = clf.score(X_valid, y_valid)

    scores.append((i,score))

100%|██████████| 1/1 [00:01<00:00,  1.28s/it]


In [24]:
scores

[(0, 0.6176470588235294)]

In [6]:
# _X_test = lil_matrix(model['vectorizer'].transform(model['df_holdout']['seq'].tolist()))
# _X_test = csr_matrix((_X_test.shape[0],_X_test.shape[1]))
# _y_test = model['df_holdout']['class'].tolist()
# model['model'].score(_X_test, _y_test)

In [7]:
# scores = []

# for i in tqdm(range(0, _X_test.shape[1], 42)):
#     X_test = _X_test.copy()
#     X_test[:,i:i+42] = 0
#     score = model['model'].score(X_test, _y_test)
#     scores.append(score)

# pd.Series(scores).value_counts()

In [8]:
# csr_matrix(np.zeros((_X_test.shape[0],_X_test.shape[1])))

In [9]:
# s.value_counts()

# print(score)

# # if model is None or score > model['score']:
# #     model = {
# #         'vectorizer': vectorizer, 
# #         'X_train': X_train, 
# #         'y_train': y_train, 
# #         'X_valid': X_valid, 
# #         'y_valid': y_valid, 
# #         'model': clf, 
# #         'score':score,
# #         'df_holdout': _df_holdout
# #     }

In [10]:
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model, f)